# Test model structures

This notebook outlines how to use the hydrologic portion of the Potions model, including setting up a model, running, and some light calibration


In [1]:
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import potions as pt

## Load data


In [2]:
data_path: str = "../input/Sleepers_Results.txt"
df = pd.read_csv(data_path, sep="\\s+", index_col="Date", parse_dates=True)
df.head()

,Qsim,Qobs,Precipitation,Temperature,AET,PET,Snow,Snowcover,SM,Recharge,SUZ,SLZ,Q0,Q1,Q2,Qsim_rain,Qsim_snow
Date,,,,,,,,,,,,,,,,,
2016-10-01,0.498270,0.138,0.0,8.7,1.29,2.27,0.0,0.0,92.7,0.0,0.0,6.3,0.0,0.0,0.498270,0.496,0.002
2016-10-02,0.461498,0.170,0.0,9.8,0.94,1.68,0.0,0.0,91.7,0.0,0.0,5.8,0.0,0.0,0.461498,0.460,0.002
2016-10-03,0.477553,0.207,4.3,11.7,1.08,1.91,0.0,0.0,94.3,0.7,0.0,6.0,0.0,0.0,0.477553,0.476,0.002
2016-10-04,0.442310,0.195,0.0,11.6,1.09,1.90,0.0,0.0,93.2,0.0,0.0,5.6,0.0,0.0,0.442310,0.441,0.002
2016-10-05,0.409667,0.163,0.0,12.1,1.30,2.30,0.0,0.0,91.9,0.0,0.0,5.1,0.0,0.0,0.409667,0.408,0.002


In [3]:
forc = pt.ForcingData(precip=df["Precipitation"], temp=df["Temperature"], pet=df["PET"])
meas_streamflow = df["Qobs"]

# Construct the model


In [4]:
mod = pt.HbvModel(lapse_rate=None)

In [5]:
res = mod.run(
    init_state=mod.default_init_state(),
    forc=forc,
    streamflow=meas_streamflow,
)

In [6]:
arr = mod.to_array()
mod2 = pt.HbvModel.from_array(arr)
res2 = mod2.run(
    init_state=mod2.default_init_state(), forc=forc, streamflow=meas_streamflow
)

In [7]:
# (
#     best_params,
#     best_res,
#     opt_res,
# ) = pt.HbvModel.simple_calibration(
#     forc=forc, meas_streamflow=meas_streamflow, metric="kge", maxiter=10, polish=True
# )

In [8]:
class ThreeLayerModel(pt.Model):
    structure = [
        [pt.SnowZone(name="snow")],
        [pt.SoilZone(name="soil")],
        [pt.GroundZoneB(name="ground")],
    ]

    def __init__(self, **kwargs) -> None:
        super().__init__(
            zones=kwargs.get("zones", dict()),
            scales=kwargs.get("scales", [1.0]),
            lapse_rates=kwargs.get("lapse_rates", []),
        )

In [9]:
mod = ThreeLayerModel()

In [10]:
mod.scales

[1.0]

In [11]:
len(mod.default_parameter_ranges(include_lapse_rates=False))

9

In [12]:
three_opt = ThreeLayerModel.simple_calibration(
    forc,
    meas_streamflow=meas_streamflow,
    metric="kge",
    use_lapse_rates=False,
    print_values=False,
    num_threads=-1,
)

{'snow_tt': (-1, 1), 'snow_fmax': (0.5, 5.0), 'soil_fc': (50, 1000), 'soil_beta': (0.5, 5.0), 'soil_k0': (0, 1.0), 'soil_lp': (0.1, 1.0), 'soil_thr': (0, 100), 'ground_k': (1e-05, 0.1), 'ground_alpha': (0.5, 3)}


/home/andrew/Documents/Research/Projects/potions/src/potions/hydro.py:312: RuntimeWarning: invalid value encountered in scalar power
  return self.k * s**self.alpha
/home/andrew/Documents/Research/Projects/potions/src/potions/hydro.py:312: RuntimeWarning: invalid value encountered in scalar power
  return self.k * s**self.alpha
/home/andrew/Documents/Research/Projects/potions/src/potions/hydro.py:312: RuntimeWarning: invalid value encountered in scalar power
  return self.k * s**self.alpha
/home/andrew/Documents/Research/Projects/potions/src/potions/hydro.py:312: RuntimeWarning: invalid value encountered in scalar power
  return self.k * s**self.alpha
/home/andrew/Documents/Research/Projects/potions/src/potions/hydro.py:312: RuntimeWarning: invalid value encountered in scalar power
  return self.k * s**self.alpha
/home/andrew/Documents/Research/Projects/potions/src/potions/hydro.py:312: RuntimeWarning: invalid value encountered in scalar power
  return self.k * s**self.alpha
/home/andr

In [13]:
three_opt[0]

{'snow_tt': np.float64(0.985611962304958),
 'snow_fmax': np.float64(2.911784129520492),
 'soil_fc': np.float64(191.2569678634385),
 'soil_beta': np.float64(1.4973735412600744),
 'soil_k0': np.float64(0.22447144578052014),
 'soil_lp': np.float64(0.26225921024554866),
 'soil_thr': np.float64(21.477659453140184),
 'ground_k': np.float64(0.07816491683285068),
 'ground_alpha': np.float64(1.123767390541841)}

## A hillslope-riparian zone model


In [14]:
class DualModel(pt.Model):
    structure = [
        [pt.SnowZone(name="snow_hs"), pt.SnowZone(name="snow_rp")],
        [pt.SoilZone(name="soil_hs"), pt.SoilZone(name="soil_rp")],
        [pt.GroundZoneB(name="ground_hs"), pt.GroundZoneB(name="ground_rp")],
    ]

    def __init__(self, **kwargs) -> None:
        super().__init__(**kwargs)


In [15]:
mod = DualModel(scales=[0.5, 0.5])
res = mod.run(forc=[forc, forc], streamflow=meas_streamflow)

In [16]:
mod.get_num_size_parameters()

1

In [17]:
mod.to_array().shape

(19,)

In [19]:
mod2 = DualModel.from_array(mod.to_array())

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 2)

In [18]:
dual_res = mod.simple_calibration(
    forc=[forc, forc],
    meas_streamflow=meas_streamflow,
    metric="nse",
    use_lapse_rates=False,
)

{'snow_hs_tt': (-1, 1), 'snow_hs_fmax': (0.5, 5.0), 'snow_rp_tt': (-1, 1), 'snow_rp_fmax': (0.5, 5.0), 'soil_hs_fc': (50, 1000), 'soil_hs_beta': (0.5, 5.0), 'soil_hs_k0': (0, 1.0), 'soil_hs_lp': (0.1, 1.0), 'soil_hs_thr': (0, 100), 'soil_rp_fc': (50, 1000), 'soil_rp_beta': (0.5, 5.0), 'soil_rp_k0': (0, 1.0), 'soil_rp_lp': (0.1, 1.0), 'soil_rp_thr': (0, 100), 'ground_hs_k': (1e-05, 0.1), 'ground_hs_alpha': (0.5, 3), 'ground_rp_k': (1e-05, 0.1), 'ground_rp_alpha': (0.5, 3), 'proportion_1': (0.1, 0.9)}


RuntimeError: The map-like callable must be of the form f(func, iterable), returning a sequence of numbers the same length as 'iterable'